- https://medium.com/optuna/lightgbm-tuner-new-optuna-integration-for-hyperparameter-optimization-8b7095e99258
- https://github.com/optuna/optuna/blob/master/examples/lightgbm_tuner_simple.py

# start

In [1]:
PRFX='0324_2'
trntmstmp=1584412344
valtmstmp=1585069785
import datetime
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)])

grand_total=1.5e8
MIN_TM_TRN=1580947200
MIN_TM_TST=1581552000
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (MIN_TM_TRN, MIN_TM_TST)])


CHNKSZ=1e6
POST_RATE_WANTED=0.1

['2020-03-17 02:32:24', '2020-03-24 17:09:45']
['2020-02-06 00:00:00', '2020-02-13 00:00:00']


# setup

In [2]:
from pathlib import Path
# import dask
# print('dask.__version__', dask.__version__)
import xgboost as xgb
# import lightgbm as lgb

import optuna
import optuna.integration.lightgbm as lgb


# optuna.logging.CRITICAL, optuna.logging.FATAL
# optuna.logging.ERROR
# optuna.logging.WARNING, optuna.logging.WARN
# optuna.logging.INFO
# optuna.logging.DEBUG
optuna.logging.set_verbosity(optuna.logging.ERROR)


# import dask_xgboost
# import dask.dataframe as dd
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve
from dask.distributed import Client
import pickle
from tqdm import tqdm
from collections import Counter
pd.set_option('display.max_rows', 500)

from functools import reduce
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
p_out=f'{HOME}/output/{PRFX}'
Path(p_out).mkdir(exist_ok=True, parents=True)

from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

# https://towardsdatascience.com/how-to-calibrate-undersampled-model-scores-8f3319c1ea5b
# How to use the function?
# Let’s say your goal is to generate a model that shows the credit default probabilities and your original 
# training data has 50,000 rows with only 500 of them labeled as target class. When you sample your non-target 
# instances randomly and reduce the total row count to 10,000, while conserving 500 target rows, our calibration
# function becomes:
# calibration(model_results, 50000, 500, 10000, 500)
# Here model_results is your model probability output array. After you train your model and put the results in it, your function is ready to use. 

def calibration(data, train_pop, target_pop, sampled_train_pop, sampled_target_pop):
    calibrated_data = \
    ((data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop)) /
    ((
        (1 - data) * (1 - target_pop / train_pop) / (1 - sampled_target_pop / sampled_train_pop)
     ) +
     (
        data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop)
     )))

    return calibrated_data

cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwng_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_fllwer_cnt',
'u2_fllwng_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]
cols_cat = ['twttyp','lang']
cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)



tgt2col=dict(zip(tgts,cols_tgt))
tgt2col

{'Retweet': 'retwt',
 'Reply': 'reply',
 'Like': 'like',
 'RTwCmnt': 'retwt_cmmnt'}

# prepare data

In [3]:
chnks_trn = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',
                    header=None,names=cols, 
                        chunksize=CHNKSZ)
# first chunk as validate data
for ichnk,df in enumerate(chnks_trn):
    df
    break

In [4]:
istrn=True
tm_min = MIN_TM_TRN if istrn else MIN_TM_TST
df['len_toks'] = df.toks.apply(len)
for media in ['Photo', 'Video', 'GIF']:
    df[f'has_media_{media}'] = df.media.fillna('').apply(lambda x: media in x)
for col in ['hshtgs', 'links', 'domns',]:
    df[f'num_{col}'] = df[col].fillna('').apply(lambda x: len(x.split('\t')) if len(x) else 0)

df['twt_age'] = df.tm - tm_min
df['u1_age']  = df.tm - df.u1_create_tm
df['u2_age']  = df.tm - df.u2_create_tm

tm_dt=pd.to_datetime(df.tm, unit='s')
df['tm_dayofweek']=tm_dt.dt.dayofweek
df['tm_hour']=tm_dt.dt.hour

df['tmdlta_u2u1']  = df.u2_create_tm - df.u1_create_tm

df['u1_fllwer_cnt_by_age'] = df.u1_fllwer_cnt / df.u1_age
df['u1_fllwng_cnt_by_age'] = df.u2_fllwng_cnt / df.u2_age

for col in ['twttyp','lang']:
    df[col]=df[col].astype('category')

if istrn: 
    df[cols_tgt]=df[cols_tgt_tmstmp].notna().astype('int8')
    df.drop(inplace=True, columns=['toks', 'hshtgs', 'media', 'links', 'domns',  
                                   'tm', 'u1_create_tm','u2_create_tm', 'u1id', 'u2id', 'twtid', ]+cols_tgt_tmstmp, )
else:
    df.drop(inplace=True, columns=['toks', 'hshtgs', 'media', 'links', 'domns', 
                                   'tm', 'u1_create_tm','u2_create_tm', 'u1id', ])   

In [5]:
df.dtypes

twttyp                  category
lang                    category
u1_fllwer_cnt              int64
u1_fllwng_cnt              int64
u1_vrfed                    bool
u2_fllwer_cnt              int64
u2_fllwng_cnt              int64
u2_vrfed                    bool
u1_fllw_u2                  bool
len_toks                   int64
has_media_Photo             bool
has_media_Video             bool
has_media_GIF               bool
num_hshtgs                 int64
num_links                  int64
num_domns                  int64
twt_age                    int64
u1_age                     int64
u2_age                     int64
tm_dayofweek               int64
tm_hour                    int64
tmdlta_u2u1                int64
u1_fllwer_cnt_by_age     float64
u1_fllwng_cnt_by_age     float64
retwt                       int8
reply                       int8
like                        int8
retwt_cmmnt                 int8
dtype: object

## prep func

In [6]:
def prp_df(df, istrn=True):
    tm_min = MIN_TM_TRN if istrn else MIN_TM_TST
    df['len_toks'] = df.toks.apply(len)
    for media in ['Photo', 'Video', 'GIF']:
        df[f'has_media_{media}'] = df.media.fillna('').apply(lambda x: media in x)
    for col in ['hshtgs', 'links', 'domns',]:
        df[f'num_{col}'] = df[col].fillna('').apply(lambda x: len(x.split('\t')) if len(x) else 0)

    df['twt_age'] = df.tm - tm_min
    df['u1_age']  = df.tm - df.u1_create_tm
    df['u2_age']  = df.tm - df.u2_create_tm

    tm_dt=pd.to_datetime(df.tm, unit='s')
    df['tm_dayofweek']=tm_dt.dt.dayofweek
    df['tm_hour']=tm_dt.dt.hour

    df['tmdlta_u2u1']  = df.u2_create_tm - df.u1_create_tm

    df['u1_fllwer_cnt_by_age'] = df.u1_fllwer_cnt / df.u1_age
    df['u1_fllwng_cnt_by_age'] = df.u2_fllwng_cnt / df.u2_age

    for col in cols_cat:
        df[col]=df[col].astype('category')

    if istrn: 
        df[cols_tgt]=df[cols_tgt_tmstmp].notna().astype('int8')
        df.drop(inplace=True, columns=['toks', 'hshtgs', 'media', 'links', 'domns',  
                                       'tm', 'u1_create_tm','u2_create_tm', 'u1id', 'u2id', 'twtid', ]+cols_tgt_tmstmp, )
    else:
        df.drop(inplace=True, columns=['toks', 'hshtgs', 'media', 'links', 'domns', 
                                       'tm', 'u1_create_tm','u2_create_tm', 'u1id', ])   
    return df

In [7]:
grand_total, grand_total/CHNKSZ

(150000000.0, 150.0)

## valid data

In [8]:
chnks_trn = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',
                    header=None,names=cols, 
                        chunksize=CHNKSZ)
# first chunk as validate data
for ichnk,df in enumerate(chnks_trn):
    print(dtnow(), 'chunk', ichnk)
#     print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') 
#            for o in (df.tm.min(), df.tm.max())])
    dfvalid = prp_df(df)
    break
print('dfvalid.shape:',dfvalid.shape)

cols_feat=[o for o in dfvalid.columns if o not in cols_tgt]

display(dfvalid[cols_feat].dtypes)

2020-03-24 19:54:02 chunk 0
dfvalid.shape: (1000000, 28)


twttyp                  category
lang                    category
u1_fllwer_cnt              int64
u1_fllwng_cnt              int64
u1_vrfed                    bool
u2_fllwer_cnt              int64
u2_fllwng_cnt              int64
u2_vrfed                    bool
u1_fllw_u2                  bool
len_toks                   int64
has_media_Photo             bool
has_media_Video             bool
has_media_GIF               bool
num_hshtgs                 int64
num_links                  int64
num_domns                  int64
twt_age                    int64
u1_age                     int64
u2_age                     int64
tm_dayofweek               int64
tm_hour                    int64
tmdlta_u2u1                int64
u1_fllwer_cnt_by_age     float64
u1_fllwng_cnt_by_age     float64
dtype: object

## trnval data func

In [9]:
def getdftrvl(tgt):
    print(tgt)
    tgtcol=tgt2col[tgt]
    chnks_trn = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',
                        header=None,names=cols, 
                            chunksize=CHNKSZ)
    len_df_wanted = int(CHNKSZ)
    # retwt          0.113031
    # reply          0.027488
    # like           0.439499
    # retwt_cmmnt    0.007742
    pos_rate_wanted = POST_RATE_WANTED
    n_pos_wanted = int(len_df_wanted*pos_rate_wanted)
    print('n_pos_wanted', n_pos_wanted)
    np.random.seed(SEED)
    lst_df = []
    n_pos_ttl = 0
    for ichnk,df in enumerate(chnks_trn):
        #skip first chunk (it was validate data)
        if ichnk==0: continue
        print(dtnow(), 'chunk', ichnk)
        df = prp_df(df)
        n_pos_ttl+= df[tgtcol].sum()
        lst_df.append(df)
        if n_pos_ttl>=n_pos_wanted: break

    df = pd.concat(lst_df)
    df.reset_index(drop=True,inplace=True)


    # https://stackoverflow.com/questions/28556942/pandas-remove-rows-at-random-without-shuffling-dataset
    idx_neg=np.where(df[tgtcol]==0)[0]
    n_neg = len(idx_neg)
    n_pos = len(df)-len(idx_neg)
    n_neg2keep = len_df_wanted-n_pos
    n_neg2rmv = n_neg-n_neg2keep
    idx_neg2rmv = np.random.choice(idx_neg, n_neg2rmv, replace=False)
    dftrvl = df.drop(idx_neg2rmv)
    dftrvl = dftrvl.sample(len(dftrvl))
    for col in cols_cat:
        dftrvl[col]=dftrvl[col].astype('category')
    
#     display(dftrvl.dtypes)
    print('dftrvl.shape:',dftrvl.shape,'dftrvl[tgtcol].mean():',dftrvl[tgtcol].mean())
    
    pops={
        'train_pop':len(df),
        'target_pop':n_pos,
        'sampled_train_pop':len_df_wanted,
        'sampled_target_pop':n_pos,
    }
    print(pops)
    return dftrvl, pops

# model

In [10]:
def train(params,dtr,dvl):
    print(params)
    best_params, tuning_history = dict(), list()
    evallist = [(dtr, 'train'), (dvl, 'eval')]
    bst = lgb.train(params=params, 
                    train_set=dtr, 
                    valid_sets=[dtr, dvl],
                    best_params=best_params,
                    tuning_history=tuning_history,
                    verbose_eval=100,
                    early_stopping_rounds=100,
                   )
                    
    return bst,best_params,tuning_history

def valid(bst,dftr,dfvl):
    prdtr = bst.predict(dftr[cols_feat],num_iteration=bst.best_iteration)
    prdvl = bst.predict(dfvl[cols_feat],num_iteration=bst.best_iteration)
    return prdtr,prdvl

def do_tgt(tgt):
    params=tgt2params[tgt]
    tgtcol=tgt2col[tgt]
    dftrvl, pops=getdftrvl(tgt)
    split=int(len(dftrvl)*0.85)
    dftr,dfvl=dftrvl[:split],dftrvl[split:]
    dtr = lgb.Dataset(dftr[cols_feat], label=dftr[tgtcol])
    dvl = lgb.Dataset(dfvl[cols_feat], label=dfvl[tgtcol])
    bst,best_params,tuning_history=train(params,dtr,dvl)
    prdtr,prdvl=valid(bst,dftr,dfvl)
    
    tgt2bst[tgt]=bst
    tgt2best_params[tgt]=best_params
    tgt2tuning_history[tgt]=tuning_history
    tgt2ytr[tgt]=dftr[tgtcol]
    tgt2yvl[tgt]=dfvl[tgtcol]
    tgt2pops[tgt]=pops
    tgt2prdtr[tgt]=prdtr
    tgt2prdvl[tgt]=prdvl


In [11]:
params_shared = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": 0,
    "boosting_type": "gbdt",
}
tgt2params = {k:params_shared for k in tgts}

tgt2bst={}
tgt2best_params={}
tgt2tuning_history={}
tgt2ytr={}
tgt2yvl={}
tgt2prdtr={}
tgt2prdvl={}
tgt2pops={}
for tgt in tgts:
    print(dtnow(), tgt, '*'*80)
    do_tgt(tgt)
    
pickle.dump(tgt2bst, open(f"{p_out}/tgt2bst.p", "wb"))

2020-03-24 19:54:06 Retweet ********************************************************************************
Retweet
n_pos_wanted 100000
2020-03-24 19:54:21 chunk 1
dftrvl.shape: (1000000, 28) dftrvl[tgtcol].mean(): 0.112823
{'train_pop': 1000000, 'target_pop': 112823, 'sampled_train_pop': 1000000, 'sampled_target_pop': 112823}
{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': 0, 'boosting_type': 'gbdt'}


/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/optuna/_experimental.py:87: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning
tune_feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.308012	valid_1's binary_logloss: 0.313027
[200]	training's binary_logloss: 0.303993	valid_1's binary_logloss: 0.311383
[300]	training's binary_logloss: 0.301065	valid_1's binary_logloss: 0.310641
[400]	training's binary_logloss: 0.298286	valid_1's binary_logloss: 0.310082
[500]	training's binary_logloss: 0.295875	valid_1's binary_logloss: 0.309765
[600]	training's binary_logloss: 0.293591	valid_1's binary_logloss: 0.309462
[700]	training's binary_logloss: 0.291297	valid_1's binary_logloss: 0.309288
[800]	training's binary_logloss: 0.289068	valid_1's binary_logloss: 0.309099
[900]	training's binary_logloss: 0.286941	valid_1's binary_logloss: 0.308958
[1000]	training's binary_logloss: 0.284835	valid_1's binary_logloss: 0.308855
Did not meet early stopping. Best iteration is:
[1000]	training's binary_logloss: 0.284835	valid_1's binary_logloss: 0.308855


tune_feature_fraction, val_score: 0.308855:  14%|#4        | 1/7 [00:35<03:31, 35.18s/it]

Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.307702	valid_1's binary_logloss: 0.312816
[200]	training's binary_logloss: 0.303753	valid_1's binary_logloss: 0.311303
[300]	training's binary_logloss: 0.30065	valid_1's binary_logloss: 0.31052
[400]	training's binary_logloss: 0.298007	valid_1's binary_logloss: 0.31009
[500]	training's binary_logloss: 0.295328	valid_1's binary_logloss: 0.309616
[600]	training's binary_logloss: 0.292884	valid_1's binary_logloss: 0.309281
[700]	training's binary_logloss: 0.290537	valid_1's binary_logloss: 0.309197
[800]	training's binary_logloss: 0.288311	valid_1's binary_logloss: 0.309065
[900]	training's binary_logloss: 0.286105	valid_1's binary_logloss: 0.308925
[1000]	training's binary_logloss: 0.283809	valid_1's binary_logloss: 0.308794
Did not meet early stopping. Best iteration is:
[1000]	training's binary_logloss: 0.283809	valid_1's binary_logloss: 0.308794


...

tune_min_child_samples, val_score: 0.295419:  60%|######    | 3/5 [01:39<01:09, 34.74s/it]

Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.297369	valid_1's binary_logloss: 0.297771
[200]	training's binary_logloss: 0.294519	valid_1's binary_logloss: 0.296786
[300]	training's binary_logloss: 0.292341	valid_1's binary_logloss: 0.296429
[400]	training's binary_logloss: 0.29034	valid_1's binary_logloss: 0.296239
[500]	training's binary_logloss: 0.288344	valid_1's binary_logloss: 0.296002
[600]	training's binary_logloss: 0.286561	valid_1's binary_logloss: 0.295853
[700]	training's binary_logloss: 0.284794	valid_1's binary_logloss: 0.295686
[800]	training's binary_logloss: 0.283124	valid_1's binary_logloss: 0.295638
[900]	training's binary_logloss: 0.281547	valid_1's binary_logloss: 0.295572
[1000]	training's binary_logloss: 0.280039	valid_1's binary_logloss: 0.295583
Did not meet early stopping. Best iteration is:
[1000]	training's binary_logloss: 0.280039	valid_1's binary_logloss: 0.295583


tune_min_child_samples, val_score: 0.295419:  80%|########  | 4/5 [02:13<00:34, 34.35s/it]

Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.29747	valid_1's binary_logloss: 0.297818
[200]	training's binary_logloss: 0.294566	valid_1's binary_logloss: 0.296785
[300]	training's binary_logloss: 0.292358	valid_1's binary_logloss: 0.296401
[400]	training's binary_logloss: 0.290292	valid_1's binary_logloss: 0.29614
[500]	training's binary_logloss: 0.288381	valid_1's binary_logloss: 0.295939
[600]	training's binary_logloss: 0.286593	valid_1's binary_logloss: 0.295806
[700]	training's binary_logloss: 0.284837	valid_1's binary_logloss: 0.295717
[800]	training's binary_logloss: 0.283157	valid_1's binary_logloss: 0.295623
[900]	training's binary_logloss: 0.281611	valid_1's binary_logloss: 0.29563
[1000]	training's binary_logloss: 0.280042	valid_1's binary_logloss: 0.295619
Did not meet early stopping. Best iteration is:
[1000]	training's binary_logloss: 0.280042	valid_1's binary_logloss: 0.295619


tune_min_child_samples, val_score: 0.295419: 100%|##########| 5/5 [02:45<00:00, 33.13s/it]


In [12]:
tgt2best_params

{'Retweet': {'lambda_l1': 3.3174170426731426,
  'lambda_l2': 2.1012711705532593e-07,
  'num_leaves': 33,
  'feature_fraction': 0.652,
  'bagging_fraction': 0.881002186360005,
  'bagging_freq': 1,
  'min_child_samples': 20},
 'Reply': {'lambda_l1': 4.337056650123979e-07,
  'lambda_l2': 1.793273131189168,
  'num_leaves': 30,
  'feature_fraction': 0.784,
  'bagging_fraction': 1.0,
  'bagging_freq': 0,
  'min_child_samples': 20},
 'Like': {'lambda_l1': 3.2429437747372396e-08,
  'lambda_l2': 1.750472601016411e-08,
  'num_leaves': 93,
  'feature_fraction': 0.9840000000000001,
  'bagging_fraction': 1.0,
  'bagging_freq': 0,
  'min_child_samples': 20},
 'RTwCmnt': {'lambda_l1': 3.125801542153872,
  'lambda_l2': 4.373108763156336,
  'num_leaves': 31,
  'feature_fraction': 0.5,
  'bagging_fraction': 1.0,
  'bagging_freq': 0,
  'min_child_samples': 20}}

# analyze

## tr vl

In [13]:
tgt2auc_tr={}
tgt2rce_tr={}
tgt2auc_vl={}
tgt2rce_vl={}
for tgt in tgt2bst:
    print(tgt)
    prdtr_i, prdvl_i = tgt2prdtr[tgt], tgt2prdvl[tgt]
    ytr_i, yvl_i = tgt2ytr[tgt], tgt2yvl[tgt]
    scr_auc_tr=compute_prauc(prdtr_i, ytr_i)
    scr_rce_tr=compute_rce(prdtr_i, ytr_i)
    scr_auc_vl=compute_prauc(prdvl_i, yvl_i)
    scr_rce_vl=compute_rce(prdvl_i, yvl_i)

    tgt2auc_tr[tgt]=scr_auc_tr
    tgt2rce_tr[tgt]=scr_rce_tr
    tgt2auc_vl[tgt]=scr_auc_vl
    tgt2rce_vl[tgt]=scr_rce_vl
    
    print('tr prauc:', f'{scr_auc_tr:.4f}','tr rce:', f'{scr_rce_tr:.4f}', )
    print('vl prauc:', f'{scr_auc_vl:.4f}','vl rce:', f'{scr_rce_vl:.4f}', )

Retweet
tr prauc: 0.4202 tr rce: 19.3046
vl prauc: 0.3307 vl rce: 13.0054
Reply
tr prauc: 0.4022 tr rce: 20.3392
vl prauc: 0.3256 vl rce: 15.5210
Like
tr prauc: 0.7919 tr rce: 24.5937
vl prauc: 0.6948 vl rce: 14.6054
RTwCmnt
tr prauc: 0.3282 tr rce: 14.5355
vl prauc: 0.2408 vl rce: 9.3780


In [14]:
lsttr=[]
lstvl=[]
for tgt in ['Retweet','Reply','Like','RTwCmnt',]:
    if tgt not in tgt2bst: continue
    lsttr+=[(f'PRAUC {tgt}',tgt2auc_tr[tgt]),
          (f'RCE {tgt}',tgt2rce_tr[tgt])]
    lstvl+=[(f'PRAUC {tgt}',tgt2auc_vl[tgt]),
          (f'RCE {tgt}',tgt2rce_vl[tgt])]

dfscrtr=pd.DataFrame(lsttr)
dfscrtr.columns=['metric','scr']
dfscrvl=pd.DataFrame(lstvl)
dfscrvl.columns=['metric','scr']
dfscr = pd.merge(dfscrtr, dfscrvl, on='metric', suffixes=('tr','vl'))
dfscr.T

,0,1,2,3,4,5,6,7
metric,PRAUC Retweet,RCE Retweet,PRAUC Reply,RCE Reply,PRAUC Like,RCE Like,PRAUC RTwCmnt,RCE RTwCmnt
scrtr,0.420239,19.3046,0.402152,20.3392,0.791944,24.5937,0.3282,14.5355
scrvl,0.330692,13.0054,0.325645,15.521,0.694811,14.6054,0.240792,9.37804


## valid

In [15]:
# tgt=tgts[1]
# tgtcol=tgt2col[tgt]
# bst=tgt2bst[tgt]

# dvalid=xgb.DMatrix(dfvalid[cols_feat], label=dfvalid[tgtcol], feature_names=cols_feat)

# prdvalid = bst.predict(dvalid, ntree_limit=bst.best_ntree_limit)

# pops=tgt2pops[tgt]

# prdvalid[:10]
# # array([0.11734424, 0.09971393, 0.05619054, 0.03059793, 0.07979691,
# #        0.01358252, 0.05293725, 0.27954698, 0.05738379, 0.01741553],
# #       dtype=float32)


# pops
# # {'train_pop': 4000000,
# #  'target_pop': 109752,
# #  'sampled_train_pop': 1000000,
# #  'sampled_target_pop': 109752}

# prdvalid_calib = calibration(prdvalid, **pops)

# prdvalid_calib[:10]
# # array([0.02952491, 0.02471944, 0.01344113, 0.00717127, 0.01945818,
# #        0.00314114, 0.0126298 , 0.08155248, 0.01373977, 0.00403964],
# #       dtype=float32)

In [16]:
def do_post_valid(tgt):
    tgtcol=tgt2col[tgt]
    bst=tgt2bst[tgt]
    pops=tgt2pops[tgt]
    prdvalid = bst.predict(dfvalid[cols_feat],num_iteration=bst.best_iteration)
    prdvalid_calib = calibration(prdvalid, **pops)
    return prdvalid,prdvalid_calib

In [17]:
tgt2yvalid={tgt:dfvalid[tgt2col[tgt]] for tgt in tgts}
tgt2prdvalid={}
tgt2prdvalid_calib={}
for tgt in tgts:
    print(dtnow(), tgt)
    tgt2prdvalid[tgt],tgt2prdvalid_calib[tgt]=do_post_valid(tgt)


2020-03-25 00:05:16 Retweet
2020-03-25 00:05:22 Reply
2020-03-25 00:05:27 Like
2020-03-25 00:05:34 RTwCmnt


In [18]:
tgt2auc_valid={}
tgt2rce_valid={}
tgt2auc_valid_calib={}
tgt2rce_valid_calib={}
for tgt in tgts:
    print(dtnow(), tgt)
    prdvalid, prdvalid_calib = tgt2prdvalid[tgt], tgt2prdvalid_calib[tgt]
    yvalid = tgt2yvalid[tgt]
    scr_auc_valid=compute_prauc(prdvalid, yvalid)
    scr_rce_valid=compute_rce(prdvalid, yvalid)
    scr_auc_valid_calib=compute_prauc(prdvalid_calib, yvalid)
    scr_rce_valid_calib=compute_rce(prdvalid_calib, yvalid)

    tgt2auc_valid[tgt]=scr_auc_valid
    tgt2rce_valid[tgt]=scr_rce_valid
    tgt2auc_valid_calib[tgt]=scr_auc_valid_calib
    tgt2rce_valid_calib[tgt]=scr_rce_valid_calib

2020-03-25 00:05:40 Retweet
2020-03-25 00:05:42 Reply
2020-03-25 00:05:43 Like
2020-03-25 00:05:45 RTwCmnt


In [19]:
for tgt in tgts:
    print(tgt)
    print('tr          prauc:', f'{tgt2auc_tr[tgt]:.4f}','tr rce:', f'{tgt2rce_tr[tgt]:.4f}', )
    print('vl          prauc:', f'{tgt2auc_vl[tgt]:.4f}','tr rce:', f'{tgt2rce_vl[tgt]:.4f}', )
    print('valid       prauc:', f'{tgt2auc_valid[tgt]:.4f}','tr rce:', f'{tgt2rce_valid[tgt]:.4f}', )
    print('valid_calib prauc:', f'{tgt2auc_valid_calib[tgt]:.4f}','tr rce:', f'{tgt2rce_valid_calib[tgt]:.4f}', )
    

Retweet
tr          prauc: 0.4202 tr rce: 19.3046
vl          prauc: 0.3307 tr rce: 13.0054
valid       prauc: 0.3227 tr rce: 12.7302
valid_calib prauc: 0.3227 tr rce: 12.7302
Reply
tr          prauc: 0.4022 tr rce: 20.3392
vl          prauc: 0.3256 tr rce: 15.5210
valid       prauc: 0.1067 tr rce: -21.7764
valid_calib prauc: 0.1067 tr rce: 11.9742
Like
tr          prauc: 0.7919 tr rce: 24.5937
vl          prauc: 0.6948 tr rce: 14.6054
valid       prauc: 0.6903 tr rce: 14.3485
valid_calib prauc: 0.6903 tr rce: 14.3485
RTwCmnt
tr          prauc: 0.3282 tr rce: 14.5355
vl          prauc: 0.2408 tr rce: 9.3780
valid       prauc: 0.0247 tr rce: -159.1833
valid_calib prauc: 0.0247 tr rce: 5.9258


In [20]:
lsttr=[]
lstvl=[]
lstvalid=[]
lstvalid_calib=[]
for tgt in ['Retweet','Reply','Like','RTwCmnt',]:
    if tgt not in tgt2bst: continue
    lsttr+=[(f'PRAUC {tgt}',tgt2auc_tr[tgt]),
          (f'RCE {tgt}',tgt2rce_tr[tgt])]
    lstvl+=[(f'PRAUC {tgt}',tgt2auc_vl[tgt]),
          (f'RCE {tgt}',tgt2rce_vl[tgt])]
    lstvalid+=[(f'PRAUC {tgt}',tgt2auc_valid[tgt]),
          (f'RCE {tgt}',tgt2rce_valid[tgt])]
    lstvalid_calib+=[(f'PRAUC {tgt}',tgt2auc_valid_calib[tgt]),
          (f'RCE {tgt}',tgt2rce_valid_calib[tgt])]

dfscrtr=pd.DataFrame(lsttr)
dfscrtr.columns=['metric','scr']
dfscrvl=pd.DataFrame(lstvl)
dfscrvl.columns=['metric','scr']
dfscrvalid=pd.DataFrame(lstvalid)
dfscrvalid.columns=['metric','scr']
dfscrvalid_calib=pd.DataFrame(lstvalid_calib)
dfscrvalid_calib.columns=['metric','scr']

dfscr = reduce(lambda df1,df2: pd.merge(df1,df2,on='metric'), 
            [dfscrtr,dfscrvl,dfscrvalid,dfscrvalid_calib])

dfscr.columns=['scr','tr','vl','valid','valid_calib']
dfscr.T

,0,1,2,3,4,5,6,7
scr,PRAUC Retweet,RCE Retweet,PRAUC Reply,RCE Reply,PRAUC Like,RCE Like,PRAUC RTwCmnt,RCE RTwCmnt
tr,0.420239,19.3046,0.402152,20.3392,0.791944,24.5937,0.3282,14.5355
vl,0.330692,13.0054,0.325645,15.521,0.694811,14.6054,0.240792,9.37804
valid,0.32271,12.7302,0.106678,-21.7764,0.690334,14.3485,0.0246643,-159.183
valid_calib,0.32271,12.7302,0.106678,11.9742,0.690334,14.3485,0.0246643,5.92585


# save

In [21]:
pickle.dump(tgt2bst, open(f"{p_out}/tgt2bst.p", "wb"))
pickle.dump(tgt2best_params, open(f"{p_out}/tgt2best_params.p", "wb"))
pickle.dump(tgt2tuning_history, open(f"{p_out}/tgt2tuning_history.p", "wb"))
pickle.dump(tgt2ytr, open(f"{p_out}/tgt2ytr.p", "wb"))
pickle.dump(tgt2yvl, open(f"{p_out}/tgt2yvl.p", "wb"))
pickle.dump(tgt2prdtr, open(f"{p_out}/tgt2prdtr.p", "wb"))
pickle.dump(tgt2prdvl, open(f"{p_out}/tgt2prdvl.p", "wb"))
pickle.dump(tgt2pops, open(f"{p_out}/tgt2pops.p", "wb"))
pickle.dump(tgt2prdvalid, open(f"{p_out}/tgt2prdvalid.p", "wb"))
pickle.dump(tgt2prdvalid_calib, open(f"{p_out}/tgt2prdvalid_calib.p", "wb"))

# infer

In [22]:
%%time
dftst=pd.read_csv(
    f'{p_in}/val_{valtmstmp}.tsv',
#     f'{p_in}/val_259A6F6DFD672CB1F883CBEC01B99F2D_1584405047.tsv',
    sep='\x01', header=None, names=cols_val,)


CPU times: user 1min 32s, sys: 9.1 s, total: 1min 41s
Wall time: 1min 41s


In [23]:
%%time
dftst = prp_df(dftst, istrn=False)

CPU times: user 51.1 s, sys: 12.5 s, total: 1min 3s
Wall time: 1min 3s


In [24]:
%%time
tgt2prdtst={}
for tgt in tgts:
    print(dtnow(), tgt)
    bst = tgt2bst[tgt]
    pops=tgt2pops[tgt]
    prdtst = bst.predict(dftst[cols_feat], num_iteration=bst.best_iteration)
    prdtst_calib = calibration(prdtst, **pops)
    tgt2prdtst[tgt] = prdtst_calib

2020-03-25 00:08:34 Retweet
2020-03-25 00:10:09 Reply
2020-03-25 00:11:26 Like
2020-03-25 00:13:06 RTwCmnt
CPU times: user 1h 55min 8s, sys: 1min 31s, total: 1h 56min 40s
Wall time: 6min 6s


In [25]:
dfsub_ids = dftst[['twtid','u2id',]]

tgt2dfsub = {}
for tgt,prdtst in tgt2prdtst.items():
    dfsub = dfsub_ids.copy()
    dfsub['scr'] = prdtst
    tgt2dfsub[tgt]=dfsub

In [26]:
%%time
for i,tgt in enumerate(['Retweet','Reply','RTwCmnt','Like',]):
    dfsub = tgt2dfsub[tgt]
    print(dtnow(), tgt)
    dfsub.to_csv(f'{p_out}/{i}_{tgt}__{valtmstmp}__{PRFX}.csv',index=False,header=False)

2020-03-25 00:14:44 Retweet
2020-03-25 00:15:48 Reply
2020-03-25 00:16:54 RTwCmnt
2020-03-25 00:18:00 Like
CPU times: user 4min 10s, sys: 9.1 s, total: 4min 19s
Wall time: 4min 20s
